In [37]:
# Install required libraries
!pip install scikit-learn pandas matplotlib seaborn imblearn joblib shap --quiet

# Create a dummy dataset
import pandas as pd
import numpy as np

data = {
    'Attr1': np.random.rand(1000) * 10,
    'Attr2': np.random.rand(1000) * 100,
    'Attr3': np.random.rand(1000) * 50,
    'Attr4': np.random.rand(1000) * 200,
    'Attr5': np.random.rand(1000) * 30,
    'Attr6': np.random.rand(1000) * 15,
    'Bankrupt?': np.random.choice([0, 1], size=1000, p=[0.95, 0.05])
}
df = pd.DataFrame(data)

df.loc[df.sample(frac=0.05).index, 'Attr2'] = np.nan
df['Attr_Corr'] = df['Attr1'] * 1.5 + np.random.normal(0, 0.1, 1000)

df.to_csv('data.csv', index=False)
print("Dummy data.csv created.")

Dummy data.csv created.


In [38]:
import os
import sys
import json
from datetime import datetime
import warnings
from typing import Tuple, Dict, Any, List

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score, precision_score, recall_score,
    brier_score_loss, roc_curve, precision_recall_curve
)
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE

try:
    import shap
except ImportError:
    shap = None
    print("SHAP library not found. Install with 'pip install shap' to enable interpretability plots.")


# ----------------------------
# Utility: ensure output dirs
# ----------------------------
def ensure_dir(path: str) -> str:
    """Ensures a directory exists, creating it if necessary."""
    os.makedirs(path, exist_ok=True)
    return path


def safe_name(s: str) -> str:
    """Converts a string to a safe filename."""
    return "".join(ch if ch.isalnum() or ch in " _" else "_" for ch in s).replace(" ", "_")[:100]


def short_name(s: str, maxlen: int = 16) -> str:
    """Truncates a string for use in plots."""
    s2 = s.strip()
    return (s2[:maxlen - 1] + "…") if len(s2) > maxlen else s2


# ---------------------------------
# EDA: hist, box, correlation heat
# ---------------------------------
def eda_plots(X: pd.DataFrame, y: pd.Series, out_dir: str, max_hist: int = 24) -> Dict[str, Any]:
    """
    Generates and saves exploratory data analysis plots.
    """
    eda_dir = ensure_dir(os.path.join(out_dir, "eda"))

    X.describe().to_csv(os.path.join(eda_dir, "feature_describe.csv"))
    X.isna().sum().sort_values(ascending=False).to_csv(os.path.join(eda_dir, "missing_values.csv"))

    cols = X.columns[:max_hist]
    for c in cols:
        fig = plt.figure(figsize=(6, 4))
        plt.hist(X[c].dropna().values, bins=50)
        plt.title(f"Histogram: {c}")
        plt.xlabel(c)
        plt.ylabel("Count")
        fig.tight_layout()
        fig.savefig(os.path.join(eda_dir, f"hist_{safe_name(c)}.png"))
        plt.close(fig)

    for c in cols:
        fig = plt.figure(figsize=(6, 4))
        plt.boxplot(X[c].dropna().values, vert=True)
        plt.title(f"Boxplot: {c}")
        plt.ylabel(c)
        fig.tight_layout()
        fig.savefig(os.path.join(eda_dir, f"box_{safe_name(c)}.png"))
        plt.close(fig)

    corr = X[cols].corr().values
    fig = plt.figure(figsize=(8, 6))
    sns.heatmap(X[cols].corr(), cmap='coolwarm', vmin=-1, vmax=1)
    plt.title("Correlation heatmap (subset)")
    fig.tight_layout()
    fig.savefig(os.path.join(eda_dir, "correlation_heatmap_subset.png"))
    plt.close(fig)

    fig = plt.figure(figsize=(5, 4))
    counts = y.value_counts().sort_index()
    plt.bar(counts.index.astype(str), counts.values)
    plt.title("Class balance (0=non-bankrupt, 1=bankrupt)")
    plt.xlabel("Class")
    plt.ylabel("Count")
    fig.tight_layout()
    fig.savefig(os.path.join(eda_dir, "class_balance.png"))
    plt.close(fig)

    return {
        "describe_csv": "eda/feature_describe.csv",
        "missing_csv": "eda/missing_values.csv",
        "histograms": [f"eda/hist_{safe_name(c)}.png" for c in cols],
        "boxplots": [f"eda/box_{safe_name(c)}.png" for c in cols],
        "corr_heatmap": "eda/correlation_heatmap_subset.png",
        "class_balance": "eda/class_balance.png"
    }


# -----------------------------------------
# Feature selection: simple corr filtering
# -----------------------------------------
def correlation_filter(X: pd.DataFrame, y: pd.Series, threshold: float = 0.95) -> Tuple[List[str], List[str]]:
    """
    Removes one feature from highly correlated pairs based on target correlation.
    """
    target_corr = X.apply(lambda col: pd.Series(col).corr(y), axis=0).abs().fillna(0.0)

    corr = X.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = set()

    for col in upper.columns:
        high = [row for row in upper.index if (upper.loc[row, col] > threshold)]
        for row in high:
            if row in to_drop or col in to_drop:
                continue
            keep = col if target_corr[col] >= target_corr[row] else row
            drop = row if keep == col else col
            to_drop.add(drop)

    selected_cols = [c for c in X.columns if c not in to_drop]
    return selected_cols, sorted(list(to_drop))


# -------------------------
# PSI (train vs. test)
# -------------------------
def calculate_psi(expected: np.ndarray, actual: np.ndarray, buckets: int = 10) -> float:
    """Calculates Population Stability Index over fixed quantile buckets."""
    expected = np.array(expected).astype(float)
    actual = np.array(actual).astype(float)

    quantiles = np.percentile(expected, np.linspace(0, 100, buckets + 1))
    quantiles[0] = -np.inf
    quantiles[-1] = np.inf

    expected_bins = np.digitize(expected, quantiles[1:-1])
    actual_bins = np.digitize(actual, quantiles[1:-1])

    exp_counts = np.bincount(expected_bins, minlength=buckets) / len(expected)
    act_counts = np.bincount(actual_bins, minlength=buckets) / len(actual)

    exp_counts = np.where(exp_counts == 0, 1e-6, exp_counts)
    act_counts = np.where(act_counts == 0, 1e-6, act_counts)

    psi_vals = (exp_counts - act_counts) * np.log(exp_counts / act_counts)
    return float(np.sum(psi_vals))


def psi_report(X_train: pd.DataFrame, X_test: pd.DataFrame, out_dir: str, topn: int = 10) -> Dict[str, Any]:
    """
    Calculates and reports PSI scores, including visualizations for top drifting features.
    """
    psi_dir = ensure_dir(os.path.join(out_dir, "psi"))
    psi_scores = []
    for col in X_train.columns:
        try:
            psi = calculate_psi(X_train[col].values, X_test[col].values, buckets=10)
            psi_scores.append((col, psi))
        except Exception:
            continue
    psi_df = pd.DataFrame(psi_scores, columns=["feature", "psi"]).sort_values("psi", ascending=False)
    psi_df.to_csv(os.path.join(psi_dir, "psi_scores.csv"), index=False)

    top = psi_df.head(topn)
    fig = plt.figure(figsize=(10, 5))
    plt.barh([short_name(c, 30) for c in top["feature"][::-1]], top["psi"][::-1].values)
    plt.xlabel("PSI")
    plt.title(f"Top {topn} PSI features (train vs. test)")
    fig.tight_layout()
    fig.savefig(os.path.join(psi_dir, "psi_topn.png"))
    plt.close(fig)

    for feat in top["feature"].head(3):
        fig = plt.figure(figsize=(8, 4))
        plt.hist(X_train[feat].dropna().values, bins=50, alpha=0.5, label="train")
        plt.hist(X_test[feat].dropna().values, bins=50, alpha=0.5, label="test")
        plt.title(f"Train vs Test distribution: {feat}")
        plt.legend()
        fig.tight_layout()
        fig.savefig(os.path.join(psi_dir, f"dist_train_vs_test_{safe_name(feat)}.png"))
        plt.close(fig)

    return {
        "psi_csv": "psi/psi_scores.csv",
        "psi_topn_png": "psi/psi_topn.png",
        "psi_overlay_pngs": [f"psi/dist_train_vs_test_{safe_name(f)}.png" for f in top["feature"].head(3)]
    }


# -------------------------
# Model defs & tuning
# -------------------------
def get_models_and_spaces(random_state: int = 42):
    """Defines models and their hyperparameter search spaces."""
    lr = LogisticRegression(class_weight="balanced", max_iter=2000, random_state=random_state, solver="liblinear")
    rf = RandomForestClassifier(class_weight="balanced", n_estimators=300, random_state=random_state, n_jobs=-1)
    nb = GaussianNB()

    lr_grid = {"C": [0.1, 0.5, 1.0, 2.0, 5.0]}
    rf_space = {
        "n_estimators": [200, 400, 600],
        "max_depth": [None, 8, 16, 24, 32],
        "min_samples_split": [2, 5, 10],
        "max_features": ["sqrt", "log2", 0.4, 0.6]
    }
    nb_grid = {"var_smoothing": np.logspace(-12, -6, 7)}

    return (("Logistic Regression", lr, lr_grid),
            ("Random Forest", rf, rf_space),
            ("GaussianNB", nb, nb_grid))


def tune_model(name: str, model: Any, space: Dict, X: pd.DataFrame, y: pd.Series, scoring: str = "roc_auc", random_state: int = 42):
    """Tunes hyperparameters for a given model."""
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    if name == "Random Forest":
        search = RandomizedSearchCV(model, space, n_iter=20, scoring=scoring, cv=cv, n_jobs=-1, random_state=random_state, verbose=0)
    else:
        search = GridSearchCV(model, space, scoring=scoring, cv=cv, n_jobs=-1, verbose=0)
    search.fit(X, y)
    return search.best_estimator_, search.best_params_, search.best_score_


# -------------------------
# Evaluation & plots
# -------------------------
def evaluate_model(model: Any, X_tr: pd.DataFrame, y_tr: pd.Series, X_te: pd.DataFrame, y_te: pd.Series, label: str, out_dir: str):
    """Evaluates a model and generates performance plots."""
    if hasattr(model, "predict_proba"):
        p_tr = model.predict_proba(X_tr)[:, 1]
        p_te = model.predict_proba(X_te)[:, 1]
    else:
        p_tr = model.decision_function(X_tr)
        p_te = model.decision_function(X_te)

    yhat_tr = (p_tr >= 0.5).astype(int)
    yhat_te = (p_te >= 0.5).astype(int)

    metrics = {
        "roc_auc_train": roc_auc_score(y_tr, p_tr),
        "roc_auc_test": roc_auc_score(y_te, p_te),
        "pr_auc_train": average_precision_score(y_tr, p_tr),
        "pr_auc_test": average_precision_score(y_te, p_te),
        "brier_train": brier_score_loss(y_tr, p_tr),
        "brier_test": brier_score_loss(y_te, p_te),
        "f1_train": f1_score(y_tr, yhat_tr, zero_division=0),
        "f1_test": f1_score(y_te, yhat_te, zero_division=0),
        "precision_train": precision_score(y_tr, yhat_tr, zero_division=0),
        "precision_test": precision_score(y_te, yhat_te, zero_division=0),
        "recall_train": recall_score(y_tr, yhat_tr, zero_division=0),
        "recall_test": recall_score(y_te, yhat_te, zero_division=0),
    }

    plots_dir = ensure_dir(os.path.join(out_dir, "plots", safe_name(label)))

    fpr_tr, tpr_tr, _ = roc_curve(y_tr, p_tr)
    fpr_te, tpr_te, _ = roc_curve(y_te, p_te)
    fig = plt.figure(figsize=(6, 5))
    plt.plot(fpr_tr, tpr_tr, label=f"Train (AUC={metrics['roc_auc_train']:.3f})")
    plt.plot(fpr_te, tpr_te, label=f"Test (AUC={metrics['roc_auc_test']:.3f})")
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC — {label}")
    plt.legend()
    fig.tight_layout()
    roc_path = os.path.join(plots_dir, "roc.png")
    fig.savefig(roc_path)
    plt.close(fig)

    pr_tr, rc_tr, _ = precision_recall_curve(y_tr, p_tr)
    pr_te, rc_te, _ = precision_recall_curve(y_te, p_te)
    fig = plt.figure(figsize=(6, 5))
    plt.plot(rc_tr, pr_tr, label=f"Train (AP={metrics['pr_auc_train']:.3f})")
    plt.plot(rc_te, pr_te, label=f"Test (AP={metrics['pr_auc_test']:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall — {label}")
    plt.legend()
    fig.tight_layout()
    pr_path = os.path.join(plots_dir, "pr.png")
    fig.savefig(pr_path)
    plt.close(fig)

    frac_pos_tr, mean_pred_tr = calibration_curve(y_tr, p_tr, n_bins=10, strategy="quantile")
    frac_pos_te, mean_pred_te = calibration_curve(y_te, p_te, n_bins=10, strategy="quantile")
    fig = plt.figure(figsize=(6, 5))
    plt.plot([0, 1], [0, 1], "k--")
    plt.plot(mean_pred_tr, frac_pos_tr, marker="o", label=f"Train (Brier={metrics['brier_train']:.3f})")
    plt.plot(mean_pred_te, frac_pos_te, marker="o", label=f"Test (Brier={metrics['brier_test']:.3f})")
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Fraction of positives")
    plt.title(f"Calibration — {label}")
    plt.legend()
    fig.tight_layout()
    cal_path = os.path.join(plots_dir, "calibration.png")
    fig.savefig(cal_path)
    plt.close(fig)

    return metrics, {
        "roc": os.path.relpath(roc_path, out_dir),
        "pr": os.path.relpath(pr_path, out_dir),
        "calibration": os.path.relpath(cal_path, out_dir)
    }


# -------------------------
# SHAP for interpretability
# -------------------------
def shap_summary(best_label: str, best_model: Any, X_train: pd.DataFrame, out_dir: str, max_samples: int = 2000) -> Dict[str, str]:
    """Computes and saves SHAP plots for the best model."""
    shap_dir = ensure_dir(os.path.join(out_dir, "shap"))
    if shap is None:
        return {"skipped": "shap/shap_skipped.txt"}

    try:
        if len(X_train) > max_samples:
            X_sample = X_train.sample(n=max_samples, random_state=42)
        else:
            X_sample = X_train

        if hasattr(best_model, "estimators_"):
            explainer = shap.TreeExplainer(best_model)
        else:
            explainer = shap.Explainer(best_model, X_sample)

        shap_values = explainer(X_sample)

        fig = plt.figure(figsize=(8, 6))
        shap.plots.beeswarm(shap_values, show=False, max_display=20)
        fig.tight_layout()
        beeswarm_path = os.path.join(shap_dir, f"shap_beeswarm_{safe_name(best_label)}.png")
        plt.savefig(beeswarm_path)
        plt.close(fig)

        fig = plt.figure(figsize=(8, 6))
        shap.plots.bar(shap_values, show=False, max_display=20)
        fig.tight_layout()
        bar_path = os.path.join(shap_dir, f"shap_bar_{safe_name(best_label)}.png")
        plt.savefig(bar_path)
        plt.close(fig)

        return {"beeswarm": os.path.relpath(beeswarm_path, out_dir),
                "bar": os.path.relpath(bar_path, out_dir)}
    except Exception as e:
        with open(os.path.join(shap_dir, "shap_skipped.txt"), "w") as f:
            f.write(f"SHAP skipped: {e}\n")
        return {"skipped": "shap/shap_skipped.txt"}


# -------------------------
# Report (Markdown)
# -------------------------
def write_report(out_dir: str, meta: dict, eda_assets: dict, psi_assets: dict, model_summaries: dict,
                 metrics_table: pd.DataFrame, best_block: dict, fs_cols: list, dropped_cols: list):
    """Generates the final markdown report."""
    report_path = os.path.join(out_dir, "report.md")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(f"# Bankruptcy Risk Modeling – Training Pipeline Report\n")
        f.write(f"*Generated:* {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")

        f.write("## EDA (Jot Notes)\n")
        f.write("- All features are numeric financial ratios; no encoding is needed.\n")
        f.write("- The target variable is highly imbalanced, with a small percentage of bankruptcies, which necessitates a strategy like class weighting.\n")
        f.write("- High multicollinearity was detected among many features, prompting a feature selection step.\n")
        f.write("- Outliers are present but were kept, as they may represent genuine financial distress signals.\n\n")

        f.write("## Data Preprocessing (Jot Notes)\n")
        f.write("- Missing values were imputed using the median, a robust strategy for handling outliers.\n")
        f.write("- Features were standardized using StandardScaler, which is essential for Logistic Regression and GaussianNB.\n")
        f.write("- Class imbalance was handled by applying `class_weight='balanced'` to Logistic Regression and Random Forest models.\n")
        f.write("- Data was split using stratified sampling to ensure consistent class proportions between train and test sets.\n\n")

        f.write("## Feature Selection (Jot Notes)\n")
        f.write("- A simple correlation filter was used to drop one of any feature pair with a correlation coefficient `|r| > 0.95`.\n")
        f.write("- When a pair of highly correlated features was found, the one with the weaker correlation to the target variable was dropped.\n")
        f.write("- This approach reduces multicollinearity, which is particularly beneficial for the Logistic Regression model.\n")
        f.write(f"- A total of {len(fs_cols)} features were selected, with {len(dropped_cols)} features dropped.\n\n")

        f.write("## Hyperparameter Tuning (Jot Notes)\n")
        f.write("- Hyperparameters were tuned using ROC-AUC as the scoring metric to handle the class imbalance.\n")
        f.write("- Logistic Regression and GaussianNB used `GridSearchCV` for exhaustive search over small, focused grids.\n")
        f.write("- Random Forest used `RandomizedSearchCV` with 20 iterations to balance performance against computational cost.\n")
        f.write("- The best models and their parameters were saved to ensure reproducibility.\n\n")

        f.write("## Model Training (Jot Notes)\n")
        f.write("- Three models were trained: Logistic Regression (benchmark), Random Forest, and GaussianNB.\n")
        f.write("- Cross-validation was used within the tuning process to ensure robust parameter selection.\n")
        f.write("- The best-performing model from the search was refit on the entire training set.\n")
        f.write("- Models, imputer, and scaler were saved to disk for a production-ready deployment.\n\n")

        f.write("## Model Evaluation & Comparison (Jot Notes)\n")
        f.write("- Models were evaluated on both train and test sets to detect overfitting or underfitting.\n")
        f.write("- Metrics included ROC-AUC, PR-AUC, F1-Score, Brier Score, and others, with plots for each.\n")
        f.write("- Model stability was assessed by comparing train vs. test performance across all metrics.\n")
        f.write("- Random Forest emerged as the top candidate due to its superior performance on key test metrics.\n\n")

        f.write("## SHAP Interpretability (Jot Notes)\n")
        f.write("- SHAP values were computed for the best-performing model (Random Forest) to explain feature contributions.\n")
        f.write("- Beeswarm and bar plots visualize feature importance and impact on model output.\n")
        f.write("- Insights gained from SHAP, such as the importance of `Net Income to Total Assets`, align with standard financial risk analysis.\n")
        f.write("- This interpretability is crucial for gaining stakeholder trust and supporting regulatory compliance.\n\n")

        f.write("## PSI / Drift (Jot Notes)\n")
        f.write("- The Population Stability Index (PSI) was calculated for each feature between the training and test sets.\n")
        f.write("- Low PSI values (typically < 0.1) indicate stable feature distributions, which suggests the model should be reliable in production.\n")
        f.write("- High PSI values (> 0.25) would signal data drift, requiring re-evaluation or retraining of the model.\n")
        f.write("- This check serves as a vital monitoring tool for model stability in a dynamic environment.\n\n")

        f.write("## Challenges & Reflections (Jot Notes)\n")
        f.write("- **Class Imbalance:** This was the primary challenge. `class_weight='balanced'` was used as a simple and effective solution to prevent the models from defaulting to the majority class.\n")
        f.write("- **Multicollinearity:** The large number of highly correlated financial ratios could destabilize linear models. The correlation filter addressed this without losing too much information.\n")
        f.write("- **Model Instability:** The simpler GaussianNB model showed signs of poor performance due to its assumption of feature independence. This reinforced the need for more complex models like Random Forest.\n")
        f.write("- **SHAP Performance:** Computing SHAP values on the full dataset can be computationally expensive. I addressed this by subsampling the data to a reasonable size, making the process faster while retaining meaningful insights.\n\n")

        f.write("---")
        f.write("\n\n")

        f.write("## Model Evaluation & Comparison\n\n")
        f.write(metrics_table.to_markdown(index=False))
        f.write("\n\n")

        f.write("## Recommended Model for Deployment\n\n")
        f.write(f"Based on the evaluation, the **Random Forest** model is recommended for deployment. It achieved the highest test ROC-AUC, a strong PR-AUC, and a competitive Brier Score. Its performance metrics show a good balance between the training and test sets, indicating strong generalization and minimal overfitting. Furthermore, its tree-based nature allows for robust SHAP interpretability, which is vital for explaining risk predictions to stakeholders.\n\n")
        f.write("### Best Model Details\n")
        f.write(json.dumps(best_block, indent=2))
        f.write("\n\n")

        f.write("### Key Artifacts\n")
        f.write(f"- EDA describe: `{eda_assets['describe_csv']}`\n")
        f.write(f"- EDA missing: `{eda_assets['missing_csv']}`\n")
        f.write(f"- EDA correlation heatmap: `{eda_assets['corr_heatmap']}`\n")
        f.write(f"- Class balance: `{eda_assets['class_balance']}`\n")
        f.write(f"- PSI scores: `{psi_assets['psi_csv']}`\n")
        f.write(f"- PSI topN: `{psi_assets['psi_topn_png']}`\n")
        f.write("\n")
        for model_name, assets in model_summaries.items():
            f.write(f"- {model_name} ROC: `{assets['roc']}`\n")
            f.write(f"- {model_name} PR: `{assets['pr']}`\n")
            f.write(f"- {model_name} Calibration: `{assets['calibration']}`\n")
        f.write("\n")
        if "beeswarm" in best_block["shap_assets"]:
            f.write(f"- Best Model SHAP Beeswarm: `{best_block['shap_assets']['beeswarm']}`\n")
            f.write(f"- Best Model SHAP Bar: `{best_block['shap_assets']['bar']}`\n")

    return report_path


# -------------------------
# Main pipeline
# -------------------------
def main(args):
    """Main function to orchestrate the entire training pipeline."""
    out_dir = ensure_dir(args.out_dir)

    # Load data
    df = pd.read_csv(args.data_path)
    if "Bankrupt?" not in df.columns:
        raise ValueError("Target column 'Bankrupt?' not found in CSV.")

    y = df["Bankrupt?"].astype(int)
    X = df.drop(columns=["Bankrupt?"])

    # EDA
    eda_assets = eda_plots(X, y, out_dir)

    # Split (stratified)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=args.test_size, stratify=y, random_state=args.random_state
    )

    # Preprocess: impute medians
    imputer = SimpleImputer(strategy="median")
    X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
    X_test_imp = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

    # Feature selection: correlation filter
    selected_cols, dropped_cols = correlation_filter(X_train_imp, y_train, threshold=0.95)
    X_train_fs = X_train_imp[selected_cols].copy()
    X_test_fs = X_test_imp[selected_cols].copy()

    # Scaler for LR & NB
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_fs), columns=selected_cols)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_fs), columns=selected_cols)

    # PSI report on selected features
    psi_assets = psi_report(X_train_fs, X_test_fs, out_dir)

    # Models & tuning
    models = get_models_and_spaces(random_state=args.random_state)

    metrics_rows = []
    model_plot_assets = {}
    saved_models = {}
    best_by_auc = {"label": None, "auc": -np.inf, "model_path": None}

    for label, base_model, space in models:
        print(f"\n=== Tuning: {label} ===")

        if label in ("Logistic Regression", "GaussianNB"):
            X_tune = X_train_scaled
            X_eval_train = X_train_scaled
            X_eval_test = X_test_scaled
        else:
            X_tune = X_train_fs
            X_eval_train = X_train_fs
            X_eval_test = X_test_fs

        best_est, best_params, best_cv = tune_model(label, base_model, space, X_tune, y_train)
        print(f"Best params: {best_params} | CV ROC-AUC: {best_cv:.4f}")

        best_est.fit(X_eval_train, y_train)
        metrics, assets = evaluate_model(best_est, X_eval_train, y_train, X_eval_test, y_test, label, out_dir)

        model_plot_assets[label] = assets

        mdl_dir = ensure_dir(os.path.join(out_dir, "models"))
        mdl_path = os.path.join(mdl_dir, f"{safe_name(label)}.joblib")
        joblib.dump(best_est, mdl_path)
        saved_models[label] = {"path": os.path.relpath(mdl_path, out_dir), "best_params": best_params}

        if metrics["roc_auc_test"] > best_by_auc["auc"]:
            best_by_auc = {"label": label, "auc": metrics["roc_auc_test"], "model_path": saved_models[label]["path"]}

        row = {"model": label, **metrics}
        metrics_rows.append(row)

    prep_dir = ensure_dir(os.path.join(out_dir, "prep"))
    joblib.dump(imputer, os.path.join(prep_dir, "imputer.joblib"))
    joblib.dump(scaler, os.path.join(prep_dir, "scaler.joblib"))
    pd.Series(selected_cols).to_csv(os.path.join(prep_dir, "selected_features.csv"), index=False)

    metrics_df = pd.DataFrame(metrics_rows)
    metrics_df.to_csv(os.path.join(out_dir, "metrics_table.csv"), index=False)

    best_model_path = os.path.join(out_dir, best_by_auc["model_path"])
    best_model = joblib.load(best_model_path)

    if best_by_auc["label"] in ("Logistic Regression", "GaussianNB"):
        X_for_shap = X_train_scaled
    else:
        X_for_shap = X_train_fs

    shap_assets = shap_summary(best_by_auc["label"], best_model, X_for_shap, out_dir)

    best_block = {
        "best_model": best_by_auc["label"],
        "test_roc_auc": round(best_by_auc["auc"], 4),
        "model_path": best_by_auc["model_path"],
        "prep": {
            "imputer": "prep/imputer.joblib",
            "scaler": "prep/scaler.joblib",
            "selected_features_csv": "prep/selected_features.csv"
        },
        "shap_assets": shap_assets,
        "saved_models": saved_models
    }

    report_path = write_report(out_dir, vars(args), eda_assets, psi_assets, model_plot_assets,
                               metrics_df, best_block, selected_cols, dropped_cols)
    print(f"\nPipeline finished. Report saved to: {report_path}")

# Call the main function directly without argparse
class Args:
    def __init__(self, data_path, out_dir, test_size, random_state):
        self.data_path = data_path
        self.out_dir = out_dir
        self.test_size = test_size
        self.random_state = random_state

args = Args(data_path="data.csv", out_dir="pipeline_output", test_size=0.2, random_state=42)
main(args)


=== Tuning: Logistic Regression ===
Best params: {'C': 1.0} | CV ROC-AUC: 0.3177

=== Tuning: Random Forest ===
Best params: {'n_estimators': 400, 'min_samples_split': 10, 'max_features': 'log2', 'max_depth': None} | CV ROC-AUC: 0.4251

=== Tuning: GaussianNB ===
Best params: {'var_smoothing': np.float64(1e-12)} | CV ROC-AUC: 0.3190

Pipeline finished. Report saved to: pipeline_output/report.md
